# Mexico City: Airbnb 2023 Analysis

by Fernando Rubi

### Dataset Information

Source: http://insideairbnb.com/get-the-data/

Additional Information: The data extracted are calendar from the date of extraction they are not an historic dataset. The Listing dataset to be used is the last available from 29-Dec-2022.

Datasets name: Calendar.csv, Listings.csv

In [ ]:
#Load Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

import datetime
from datetime import timedelta

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import AdaBoostClassifier
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

import folium
from folium.plugins import MarkerCluster
from folium.features import DivIcon

## Loading the calendar datasets

#### Load the Calendar Datasets and create a new merged dataframe

In [ ]:
#Load calendar datasets
calendar1 = pd.read_csv('01_26Mar22/calendar.csv.gz', compression='gzip')
calendar2 = pd.read_csv('02_21Jun22/calendar.csv.gz', compression='gzip')
calendar3 = pd.read_csv('03_22Sept22/calendar.csv.gz', compression='gzip')
calendar4 = pd.read_csv('04_29Dec22/calendar.csv.gz', compression='gzip')

In [ ]:
cal_lst = [calendar1, calendar2, calendar3, calendar4]

In [ ]:
i=0
for cal in cal_lst:
    i+=1
    #Change the date column from string to datetime format.
    cal['date'] = pd.to_datetime(cal['date'], format='%Y-%m-%d')
    min_date = cal['date'].dt.date.min()
    max_date = cal['date'].dt.date.max()
    
    #Print the number of rows and date range from each dataframe
    print(f"Calendar{i}: {cal.shape[0]} rows; date range from {min_date} to {max_date}")
    
    #Create the variable booked inverse to column available
    cal['booked'] = cal['available'].replace({'f':1, 't':0})
    
    #Create new columns for the Year, Month and Weekday
    cal['year'] = cal['date'].dt.year
    cal['month'] = cal['date'].dt.month
    cal['weekday'] = cal['date'].dt.weekday #0: Monday,... , 6:Sunday


calendar1.head()

## Question 1: How many days before a main holiday should I reserve to still have several options?

### Occupancy Rate Formula

Occupancy Rate = Rented days / Total Available days

In [ ]:
def graph_cal_grouped(df, group, y, title='Calendar Bar Plot', annotation='Grouped mean'):
    '''
    INPUT
    df: dataframe to group and plot
    group: column name as string to use for group 
    y: column name as string to use for Y axis
        
    OUTPUT
    Graph object    
    '''
    df_grp = df.groupby(group)[y].mean().to_frame()

    fig = px.bar(
        df_grp,
        y=y,
        color=y,
        title=f'<span style="color:#4169e1">{title}</span>',
        range_color=[0,1],
        color_continuous_scale='inferno' #color options: bluered, inferno, viridis
    ) 
    
    fig.update_layout(yaxis_range=[0,0.8])

    fig.add_annotation(
        text = annotation,
        xref = 'paper',
        yref = 'paper',
        x = -0.04,
        y = 1.13,
        showarrow = False,
        align = 'left',
        xanchor = 'left',
        font = {'size': 11}
    )

    return fig

In [ ]:
i=1
for cal in cal_lst:
    title = 'Calendar{} Bar Plot'.format(i)
    annotation = 'Daily occupancy rate mean'
    fig = graph_cal_grouped(cal, 'date', 'booked', title, annotation)
    fig.show()
    i+=1

In [ ]:
day_str = '2022-10-29'

for cal in cal_lst:
    cal_min = cal['date'].dt.date.min()
    cal_month = cal_min.month
    holiday = datetime.datetime.strptime(day_str, '%Y-%m-%d').date()
    
    if cal_month > 11:
        holiday = holiday+timedelta(days=364)
    
    else:
        pass

    delta_days = holiday - cal_min
    
    df_holiday = cal.groupby('date')['booked'].mean().to_frame()
    occupancy_val = round((df_holiday.loc[str(holiday),'booked'] *100),2)
        
    
    print('Check date:{} / Holiday date:{} / Days Difference:{} / mean occupancy:{}%'.format(
        cal_min, holiday, delta_days.days, occupancy_val)
         )

## Question 2: If I want to try to avoid crowded places or traffic jams on the highway, which day of the week should I travel?

In [ ]:
section1 = str(calendar2['date'].dt.date.min())
section2 = str(calendar3['date'].dt.date.min())
section3 = str(calendar4['date'].dt.date.min())
section4 = str(calendar1['date'].dt.date.max()+timedelta(days=1))

In [ ]:
section_lst = [section1, section2, section3, section4]

In [ ]:
def merge_cal(cal, sec):
    '''
    INPUT
    cal: calendar dataframe in a list
    sec: strings with the cut dates in a list
    
    OUTPUT
    df: merged dataframe
    '''
    tmp = ['tmp1', 'tmp2', 'tmp3', 'tmp4']
    merge = {}
    
    for i in range(4):
        merge[tmp[i]] = cal[i].loc[cal[i]['date']<sec[i]]
        i+=1
    
    df = pd.concat(list(merge.values()))
    
    return df

In [ ]:
df_calendar = merge_cal(cal_lst, section_lst)

min_date = df_calendar['date'].dt.date.min()
max_date = df_calendar['date'].dt.date.max()
print(f"Merged calendar range from {min_date} to {max_date}")

In [ ]:
title = '<span style="color:#4169e1">Merged Calendar Dataframe</span> <span style="color:#808080">Daily mean occupancy</span>'
annotation = 'Data is from March 26th, 2022 to March 26th, 2023'
fig = graph_cal_grouped(df_calendar, 'date', 'booked', title, annotation)

i=0
for s in section_lst:
    i += 1
    fig.add_vline(x=s, line_width=1, line_dash="dash", line_color="red", opacity=.5)
    fig.add_annotation(x=s, y=.75, text='<b>X</b>', showarrow=False, hovertext=f'cut #{i} date: '+s)

fig.show()

In [ ]:
df_daily = df_calendar.groupby('date')['booked'].mean().to_frame()
df_daily = df_daily.reset_index()
df_daily['weekday'] = df_daily['date'].dt.weekday
df_daily['pct_change'] = df_daily['booked'].pct_change()
df_daily['abs_change'] = df_daily['pct_change'].abs()
df_daily.head()

In [ ]:
title = '<span style="color:#4169e1">Calendar Dataframe</span> <span style="color:#808080">Weekday mean occupancy</span>'
annotation = 'Days of the week starting monday with 0 and ending Sunday with 6'
fig = graph_cal_grouped(df_daily, 'weekday', 'abs_change', title, annotation)
fig.update_layout(yaxis_range=[0,0.1], coloraxis=dict(cmin=0, cmax=0.1))
fig.show()

In [ ]:
title = '<span style="color:#4169e1">Calendar Dataframe</span> <span style="color:#808080">Weekday mean occupancy</span>'
annotation = 'Days of the week starting monday with 0 and ending Sunday with 6'
fig = graph_cal_grouped(df_calendar, 'weekday', 'booked', title, annotation)
fig.update_layout(yaxis_range=[0,0.4], coloraxis=dict(cmin=0.3, cmax=0.4))
fig.show()

## Load the Listings Dataset

In [ ]:
#Load listings dataset
df_listings = pd.read_csv('04_29Dec22/listings.csv.gz', compression='gzip')
rows = df_listings.shape[0]
columns = df_listings.shape[1]
print("Listings rows: {}, columns: {}".format(rows, columns))

In [ ]:
#Calculate the occupancy rate per listing
occupancy_rate = df_calendar.groupby(['listing_id'])['booked'].mean().to_frame()
occupancy_rate.describe()

## Occupancy Rate Distribution

### Classification of High - Medium - Low Occupancy Rate

In [ ]:
occupancy_rate.quantile([0.33,0.67])

In [ ]:
perc_low = np.percentile(occupancy_rate,33)
perc_high = np.percentile(occupancy_rate,67)
print('Range: from {:.3f} to {:.3f}'.format(perc_low, perc_high))

In [ ]:
bins = [0, perc_low, perc_high, 1]
names = ['Low', 'Medium', 'High']

occupancy_rate['occup_cat'] = pd.cut(occupancy_rate['booked'], bins, labels=names, include_lowest=True)
occupancy_rate.head()

In [ ]:
title = '<span style="color:#4169e1">Distribution & Box Plot</span> <span style="color:#808080">Listings mean occupancy</span>'
annotation = 'Calculated yearly occupancy per listing'
fig = px.histogram(occupancy_rate,
                   x='booked',
                   marginal='box',
                   #color='occup_cat',
                   title=title,
                   )
fig.add_annotation(
        text = annotation,
        xref = 'paper',
        yref = 'paper',
        x = -0.07,
        y = 1.13,
        showarrow = False,
        align = 'left',
        xanchor = 'left',
        font = {'size': 11})


for i in range(1,3):
    fig.add_vline(x=bins[i], line_width=1, line_dash="dash", line_color="red", opacity=.5)
    fig.add_annotation(x=bins[i], y=2500, text='<b>X</b>', showarrow=False,
                   hovertext='occupancy rate: '+str(format(bins[i]*100,'.2f'))+'%')

fig.show()

In [ ]:
occupancy_rate['occup_cat'].value_counts()

In [ ]:
title = '<span style="color:#4169e1">Listings Box Plot</span> <span style="color:#808080">Occupancy rate categorized</span>'
annotation = 'Categories: Low: 0 - 0.098 / Medium: 0.098 - 0.4 / High: 0.4 - 1'
fig = px.box(occupancy_rate, x='occup_cat', y='booked', color='occup_cat',
             title=title,
             category_orders = {'occup_cat':['Low','Medium','High']},
             color_discrete_map = {'Low':'#d62728', 'Medium':'#bcbd22', 'High':'#2ca02c'})
fig.add_annotation(
        text = annotation,
        xref = 'paper',
        yref = 'paper',
        x = -0.07,
        y = 1.13,
        showarrow = False,
        align = 'left',
        xanchor = 'left',
        font = {'size': 11})

fig.show()

### Question 3: What neighborhood is the best for having an Airbnb in Mexico City?

In [ ]:
#Add the dataframe occupancy rate to the dataframe listings
listings = pd.merge(left=df_listings, right=occupancy_rate, left_on='id', right_index=True)

In [ ]:
neighb_cnt = listings.groupby(['neighbourhood_cleansed','occup_cat'])['booked'].count().to_frame()
neighb_cnt['perc'] = neighb_cnt.groupby('neighbourhood_cleansed')['booked'].transform(lambda x:100* x/x.sum())
neighb_cnt = neighb_cnt.reset_index()

title = '<span style="color:#4169e1">Neighborhood Bar Plot</span> <span style="color:#808080">Listings count</span>'
annotation = 'Order by total descending count per neighborhood'

fig = px.bar(neighb_cnt,
             x='neighbourhood_cleansed',
             y='booked',
             color='occup_cat',
             text=neighb_cnt['perc'].apply(lambda x: '{0:1.2f}%'.format(x)),
             title=title,
             category_orders = {'occup_cat':['Low','Medium','High']},
             color_discrete_map = {'Low':'#ff6961', 'Medium':'#ffd700', 'High':'#3cb371'})
            

fig.add_annotation(
        text = annotation,
        xref = 'paper',
        yref = 'paper',
        x = -0.07,
        y = 1.13,
        showarrow = False,
        align = 'left',
        xanchor = 'left',
        font = {'size': 11})

fig.update_layout(barmode='stack', xaxis={'categoryorder':'total descending'})

fig.show()

In [ ]:
#Convert price string to value
listings['price_value'] = listings['price'].replace('\$|,', '', regex=True).astype('float')
listings['price_value'].describe()

In [ ]:
#Plot correlation price per neighborhood
neighb_price = listings.groupby(['neighbourhood_cleansed','occup_cat'])['price_value'].quantile(.50).to_frame()
neighb_price = neighb_price.reset_index()


title = '<span style="color:#4169e1">Price per Neighborhood</span> <span style="color:#808080">Median value per category</span>'
annotation = 'Order by total descending count per neighborhood'

fig = px.bar(neighb_price,
             x='neighbourhood_cleansed',
             y='price_value',
             color='occup_cat',
             barmode='group',
             #text_auto=True,
             title=title,
             category_orders = {'occup_cat':['Low','Medium','High']},
             color_discrete_map = {'Low':'#ff6961', 'Medium':'#ffd700', 'High':'#3cb371'})
            

fig.add_annotation(
        text = annotation,
        xref = 'paper',
        yref = 'paper',
        x = -0.07,
        y = 1.13,
        showarrow = False,
        align = 'left',
        xanchor = 'left',
        font = {'size': 11})

fig.update_layout(xaxis={'categoryorder':'total descending'})

fig.show()

In [ ]:
#Plot the folium map with all the available houses marked by their occupancy classification
cdmx_coordinate = [19.43685331156142, -99.12272036926497]
cdmx_map = folium.Map(location=cdmx_coordinate, zoom_start=11)

In [ ]:
markers = listings[['id', 'latitude', 'longitude', 'neighbourhood_cleansed', 'occup_cat']]

In [ ]:
markers['color'] = markers['occup_cat'].replace({'Low':'red','Medium':'lightgreen','High':'green'})
markers.head()

In [ ]:
#Create feature groups
High = MarkerCluster(name='High').add_to(cdmx_map)
Medium = MarkerCluster(name='Medium').add_to(cdmx_map)
Low = MarkerCluster(name='Low').add_to(cdmx_map)

#Add Layer controls
folium.LayerControl().add_to(cdmx_map)

#Create and add each marker to their occupancy groups
for index, record in markers.iterrows():
    coordinate=[record[1],record[2]]
    result=record[5]
    marker = folium.Marker(coordinate, icon=folium.Icon(color='black',icon_color=result))
    
    if record[4] == 'High':
        High.add_child(marker)
        
    elif record[4] == 'Medium':
        Medium.add_child(marker)
    
    else:
        Low.add_child(marker)

In [ ]:
cdmx_map

In [ ]:
#cdmx_map.save('cdmx_map.html')

## Question 4: Does the size matter? What are the most important features to have a high occupancy?

In [ ]:
listings.info()

#### Preprocessing dataframe

In [ ]:
#Clean data frame, drop columns with no data or duplicated data
drop_list = ['listing_url', 'scrape_id', 'last_scraped', 'source', 'name', 'description',
             'neighborhood_overview', 'picture_url', 'host_url', 'host_name', 'host_since',
             'host_location', 'host_about', 'host_thumbnail_url', 'host_picture_url',
             'host_neighbourhood', 'host_listings_count', 'host_total_listings_count',
             'host_verifications', 'neighbourhood', 'neighbourhood_group_cleansed', 'calendar_updated',
             'has_availability', 'availability_30', 'availability_60', 'availability_90',
             'availability_365', 'calendar_last_scraped', 'number_of_reviews_ltm', 'number_of_reviews_l30d',
             'first_review', 'last_review', 'license', 'calculated_host_listings_count',
             'calculated_host_listings_count_entire_homes', 'calculated_host_listings_count_private_rooms',
             'calculated_host_listings_count_shared_rooms', 'reviews_per_month', 'bathrooms', 'property_type', 'amenities',
             'minimum_nights', 'maximum_nights', 'minimum_minimum_nights', 'minimum_maximum_nights',
             'maximum_minimum_nights', 'maximum_maximum_nights', 'price']

In [ ]:
listings_filtered = listings.drop(drop_list, axis=1)

In [ ]:
#Remove symbols from columns and change to float
perc_col = ['host_response_rate', 'host_acceptance_rate']
for col in perc_col:
    listings_filtered[col] = listings_filtered[col].str.extract('(\d*)').astype('float')

listings_filtered[perc_col].head()

In [ ]:
#Extract values from text column in a new column
listings_filtered['bathrooms'] = listings_filtered['bathrooms_text'].str.extract('(\d*\.*\d+)').astype('float')
listings_filtered['bathrooms'] = listings_filtered['bathrooms'].fillna(0.5)


listings_filtered[listings_filtered['bathrooms'] == 9.5][['bathrooms_text','bathrooms']]

In [ ]:
listings_filtered = listings_filtered.drop('bathrooms_text', axis=1)
listings_filtered.describe(include='object')

In [ ]:
def binary_columns(df,col_lst):
    for col in col_lst:
        df[col+'_bin']=df[col].replace({'t':1, 'f':0, np.nan:0})
    df.drop(labels=col_lst, axis=1, inplace=True)
    return df

In [ ]:
bin_list = ['host_is_superhost', 'host_has_profile_pic', 'host_identity_verified', 'instant_bookable']

In [ ]:
listings_filtered = binary_columns(listings_filtered,bin_list)

In [ ]:
dummies_list = list(listings_filtered.describe(include='object').columns)

In [ ]:
listings_dummies = pd.get_dummies(listings_filtered, prefix=dummies_list, columns=dummies_list)

In [ ]:
na_list = list(listings_dummies.loc[:, listings_dummies.isna().any()].columns)

In [ ]:
def imputing_cat_na(df, col_lst):
    '''
    Fill NA rows with mean by occupancy category
    
    INPUT
    df: dataframe with NA values and column with 'occup_cat' category column
    col_lst: column list to fill with mean value
    
    OUTPUT
    df: dataframe with NA values filled with mean by category
    
    '''
    for col in col_lst:
        mean = df.groupby(['occup_cat'])[col].mean()
        low = mean[0]
        medium = mean[1]
        high = mean[2]

        df.loc[df['occup_cat']=='Low', col] = df[col].replace(np.nan,low)
        df.loc[df['occup_cat']=='Medium', col] = df[col].replace(np.nan,medium)
        df.loc[df['occup_cat']=='High', col] = df[col].replace(np.nan,high)

    return df

In [ ]:
listings_dummies = imputing_cat_na(listings_dummies, na_list)

In [ ]:
# Drop identifiers and occupancy rate value
listings_corr = listings_dummies.drop(labels=['id', 'host_id','latitude', 'longitude', 'booked'], axis=1)

# Convert categorical variable to numerical variable for analysis
listings_corr['occup_cat'] = listings_dummies['occup_cat'].replace({'Low':0, 'Medium':1, 'High':2}).astype('int')

In [ ]:
#Calculating Correlation 
correlation=listings_corr.corr()

most_related=correlation.index[abs(correlation['occup_cat'])>0.1]

print(most_related, len(most_related))

#Plotting the Correlation in HeatMap for the data columns which has correlation value more than 0.1
plt.figure(figsize=(12,12))

Corr_Heatmap=sns.heatmap(listings_corr[most_related].corr(),annot=True,cmap="GnBu")

In [ ]:
#Droping labels and separating predictors
x = listings_dummies.drop(labels=['id', 'host_id','latitude', 'longitude', 'booked', 'occup_cat'], axis=1)

#or use only positive related columns with booked
#x = listings_dummies[most_related]

# Transforming predictor categorical labels into numerical labels????
y = listings_dummies['occup_cat']

In [ ]:
transform = preprocessing.StandardScaler()
X = transform.fit(x).transform(x)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)

In [ ]:
print('Train shape: ',X_train.shape)
print('Test shape: ',X_test.shape)

#### AdaBoost Classifier

In [ ]:
model_ada = AdaBoostClassifier(learning_rate=1)
model_ada.fit(X_train, y_train)
y_pred = model_ada.predict(X_test)

In [ ]:
print('Accuracy Score: ', accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

### Hyperparameter tuning using GridSearch

In [ ]:
ada_clf = AdaBoostClassifier()
params= {
    'n_estimators': np.arange(100,300,10),
    'learning_rate': [0.1, 1, 2]
}
classes = y_train.unique()

In [ ]:
grid_clf = GridSearchCV(estimator=ada_clf, scoring='f1_weighted', param_grid=params, cv=5)

grid_clf.fit(X_train, y_train)

In [ ]:
print('best parameters: ', grid_clf.best_params_)

In [ ]:
ada_c = AdaBoostClassifier(**grid_clf.best_params_)
ada_c.fit(X_train, y_train)

y_pred = ada_c.predict(X_test)

In [ ]:
print('Accuracy Score: ', accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

In [ ]:
cm = confusion_matrix(y_test, y_pred)

## Get Class Labels
labels = model_ada.classes_

In [ ]:
fig = plt.figure(figsize=(8, 7))
ax= plt.subplot()

sns.heatmap(cm, annot=True, ax = ax, vmin=200, vmax=1000, center=700 , fmt = 'g', cmap='inferno')

ax.set_xlabel('Predicted Label', fontsize=25)
ax.xaxis.set_label_position('bottom')
ax.xaxis.set_ticklabels(labels, fontsize = 12)
ax.xaxis.tick_bottom()

ax.set_ylabel('True Label', fontsize=25)
ax.yaxis.set_ticklabels(labels, fontsize = 12)
plt.yticks(rotation=0)

In [ ]:
#get feature importances from the model
headers = ['features', 'score']
values = sorted(zip(x.columns, model_ada.feature_importances_), key=lambda x: x[1] * -1)
ada_feature_importances = pd.DataFrame(values, columns = headers)

In [ ]:
features = ada_feature_importances['features'][:15]
y_pos = np.arange(len(features))
scores = ada_feature_importances['score'][:15]

In [ ]:
plt.figure(figsize=(10,5))
plt.bar(y_pos, scores, align='center', alpha=0.5)
plt.xticks(y_pos, features, rotation='vertical')
plt.xlabel('Features')
plt.ylabel('Score')
plt.title('Feature importances')
 
plt.show()

#### Additional Sources:

Occupancy Rate: https://www.wallstreetmojo.com/occupancy-rate/